In [1]:
import pandas as pd 
import datetime
import csv
import os 
import requests 
import datetime as dt
import time

# Exercise

For your exercise do the following:

1. Choose a reddit page you want to crawl
2. The following fields should be present when you crawl **(10 points)**:
    - author
    - subreddit
    - date created 
    - number of comments 
    - score
    - submission title 
    - submission description
3. After crawling, save your results to a pandas dataframe **(3 points)**. 
4. Answer the following questions **(12 points)**:
    - How many submissions were you able to gather? 
    - Who has the most submissions? 
    - Which submission has the highest score? 
    - Which submission has the highest number of comments?
    - Which day of the week has the most submissions? 
    
**Tip:** _For item#4, recall how to use the aggregation functions in `pandas` like count, value_counts, sum, etc. For getting the day of the week, look into how to get the `dayofweek` from a datetime object in `pandas`. (Hint: You may need to use `pd.to_datetime` to convert your date column...)_

In [6]:
def to_utc(date):
    #This function converts an object to UTC. This is to automate the conversion 
    #of dates instead of going to https://www.unixtimeconverter.io/ 
    return int(date.replace(tzinfo=dt.timezone.utc).timestamp())
    
def to_readable_date(timestamp):
    #This function converts the UTC format to a Year-Month-Day format 
    return dt.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d")


#Declare start and end of reddit posts to extract 
start_date = dt.datetime.strptime("2020-03-01", "%Y-%m-%d")
end_date = dt.datetime.strptime("2020-05-31", "%Y-%m-%d")

#Create a range of dates to iterate 
#Note: Periods here represents the number of days it will create from the start date 
#We also do a +2 since it will only generate up to April 29. We inlcude May 1 
#since we want to get data from the last day which is April 30 to May 1 
date_range = (pd.date_range(
                start_date, 
                periods=(end_date - start_date).days + 2)
              .tolist())


#prepare the parameters needed to call the API
sort_type="score"
sort="desc"
fields=["id","title","selftext","score","num_comments","created_utc","author","subreddit"]
subreddit = 'urbanplanning'
url = "https://api.pushshift.io/reddit/submission/search/"
results = []
#loop through the dates 
for i, s_date in enumerate(date_range):
    #prevents us from getting an index out of range error
    if i != len(date_range)-1:
        #declare end date 
        e_date = date_range[i+1]
        #call the API
        r = requests.get(url = url, params={
            'after': to_utc(s_date),
            'before': to_utc(e_date),
            'sort_type': sort_type,
            'sort': sort,
            'subreddit': subreddit,
            'fields': fields,
            "size": 500
        })

        #add logs 
        print(f"Doing {s_date.strftime('%Y-%m-%d')} to {e_date.strftime('%Y-%m-%d')}")
        if r.status_code == 200:
            results.append(r.json()['data'])
            print("=====Done")
        else:
            print("=====Skipped")
        #so that we dont get blocked from abusing the API we call it after pausing for 1 second
        time.sleep(1)
        
        
     

Doing 2020-03-01 to 2020-03-02
=====Done
Doing 2020-03-02 to 2020-03-03
=====Done
Doing 2020-03-03 to 2020-03-04
=====Done
Doing 2020-03-04 to 2020-03-05
=====Done
Doing 2020-03-05 to 2020-03-06
=====Done
Doing 2020-03-06 to 2020-03-07
=====Done
Doing 2020-03-07 to 2020-03-08
=====Done
Doing 2020-03-08 to 2020-03-09
=====Done
Doing 2020-03-09 to 2020-03-10
=====Done
Doing 2020-03-10 to 2020-03-11
=====Done
Doing 2020-03-11 to 2020-03-12
=====Done
Doing 2020-03-12 to 2020-03-13
=====Done
Doing 2020-03-13 to 2020-03-14
=====Done
Doing 2020-03-14 to 2020-03-15
=====Done
Doing 2020-03-15 to 2020-03-16
=====Done
Doing 2020-03-16 to 2020-03-17
=====Done
Doing 2020-03-17 to 2020-03-18
=====Done
Doing 2020-03-18 to 2020-03-19
=====Done
Doing 2020-03-19 to 2020-03-20
=====Done
Doing 2020-03-20 to 2020-03-21
=====Done
Doing 2020-03-21 to 2020-03-22
=====Done
Doing 2020-03-22 to 2020-03-23
=====Done
Doing 2020-03-23 to 2020-03-24
=====Done
Doing 2020-03-24 to 2020-03-25
=====Done
Doing 2020-03-25

In [7]:
flat_list = []
#loop through the reddit results
for sublist in results:
    #check if sublist is not empty. The reason we have empty lists is because there are days wherein there are no submissions
    if sublist is not None:
        #for each dictionary in the sublist add it to the flat list 
        for item in sublist:
            flat_list.append(item)

#pandas has a useful function called from_dict which will convert a list of dictionary objects into a dataframe
df = pd.DataFrame.from_dict(flat_list)
display(df.head())
df.to_csv("reddit_urbanplanning.csv")
df

,author,created_utc,id,num_comments,score,selftext,subreddit,title
0,TransportationHub456,1583031144,fbnr0y,1,1,,urbanplanning,2020 New Flyer XD60s On The Q52/Q53/LGA Link Q...
1,SuccessfulOperation,1583091543,fbzbey,15,1,,urbanplanning,Did pedestrian malls ruin U.S. downtowns?
2,gh0bs,1583094674,fc04fb,1,1,,urbanplanning,Whoever came up with this concept of “it’s lik...
3,Mynameis__--__,1583039795,fbpfee,0,1,,urbanplanning,"Ghetto: The Invention of a Place, the History ..."
4,Nick-Anand,1583037793,fbp25u,10,1,,urbanplanning,Automobile costs represent $20k per family ann...


,author,created_utc,id,num_comments,score,selftext,subreddit,title
0,TransportationHub456,1583031144,fbnr0y,1,1,,urbanplanning,2020 New Flyer XD60s On The Q52/Q53/LGA Link Q...
1,SuccessfulOperation,1583091543,fbzbey,15,1,,urbanplanning,Did pedestrian malls ruin U.S. downtowns?
2,gh0bs,1583094674,fc04fb,1,1,,urbanplanning,Whoever came up with this concept of “it’s lik...
3,Mynameis__--__,1583039795,fbpfee,0,1,,urbanplanning,"Ghetto: The Invention of a Place, the History ..."
4,Nick-Anand,1583037793,fbp25u,10,1,,urbanplanning,Automobile costs represent $20k per family ann...
...,...,...,...,...,...,...,...,...
1523,toxicbrew,1590905474,gttt8c,19,1,I'm just curious how exactly one would go and ...,urbanplanning,How would you redevelop the typical American s...
1524,A9M4D,1590928378,gtyag7,22,1,I’m a total newbie to urban planning and real ...,urbanplanning,Are cities becoming the same all around the wo...
1525,elfriedemfcorel,1590965046,gu8oru,0,1,,urbanplanning,Video Chat Rooms - Free Random Webcam Chats
1526,Jalal-ud-deeeen,1590961203,gu7kyg,7,1,Cities like Lahore\n\n[View Poll](https://www....,urbanplanning,For a mega city in a developing world without ...


In [11]:
#1.0 Total number of submissions gathered?
index = df.index
submissions = len(index)
print('1.0) Total Number of Submissions:',submissions)

1.0) Total Number of Submissions: 1528


In [25]:
#2.0 Who has the most submissions?

Author_mode = df.author.mode()

df['author'].value_counts()
Authorcount= df['author'].value_counts()

print('2.0) Author with most number of submissions:',Author_mode[0]," with ",Authorcount[0], "total submissions.")

2.0) Author with most number of submissions: Andrew_Korenchkin  with  62 total submissions.


In [29]:
#3.0 Which submission has the highest score?

highscoreindex=df['score'].argmax()
#highscoreindex

print(df.loc[highscoreindex])

author                                                RChickenMan
created_utc                                            1586813634
id                                                         g0s5p5
num_comments                                                  247
score                                                         690
selftext        I feel like the people who fetishise [tiny hou...
subreddit                                           urbanplanning
title           Should we tell the Americans who fetishise "ti...
Name: 641, dtype: object


In [30]:
#4.0 Which submission has the highest number of comments?

highcommentsindex=df['num_comments'].argmax()
#highcommentsindex

print(df.loc[highcommentsindex])

author                                                RChickenMan
created_utc                                            1586813634
id                                                         g0s5p5
num_comments                                                  247
score                                                         690
selftext        I feel like the people who fetishise [tiny hou...
subreddit                                           urbanplanning
title           Should we tell the Americans who fetishise "ti...
Name: 641, dtype: object


In [84]:
#5.0 Which day of the week has the most submissions

from datetime import datetime
from datetime import date 
import calendar 
from pprint import pprint


def to_readable_date(timestamp):
    #This function converts the UTC format to a Year-Month-Day format 
    return dt.datetime.fromtimestamp(timestamp).strftime("%Y-%m-%d")

#extracting utc date of creation into a separate list
utcs = df['created_utc'].to_list()

#converting the utcs list into integers
utcsint = [int(i) for i in utcs] 

#utcsint

#runthrough conversion of UTC into date stamps
converted = []

for i in utcsint:
    converted.append(to_readable_date(i))

#print (converted)

#switch of date formats into strings

dates = (datetime.strptime(ts, '%Y-%m-%d') for ts in converted)
date_strings = [datetime.strftime(d, '%m-%d-%Y') for d in dates]

#pprint(date_strings)

df["Dates"] = date_strings

#df

def findDay(date): 
    dayname = datetime.strptime(date, '%Y-%m-%d').weekday() 
    return (calendar.day_name[dayname]) 

daylist = []

for i in converted:
    daylist.append(findDay(i))

#daylist

df['Posting_Day'] = daylist

#df

Day_mode = df.Posting_Day.mode()

df['Posting_Day'].value_counts()
Daycount= df['Posting_Day'].value_counts()

print('5.0) Day of the week with most number of submissions:',Day_mode[0]," with ",Daycount[0], "total submissions.")



5.0) Day of the week with most number of submissions: Thursday  with  249 total submissions.
